
*   Computing Platforms: Set up the Workspace for Machine Learning Projects.  https://ms.pubpub.org/pub/computing
*  Machine Learning for Predictions. https://ms.pubpub.org/pub/ml-prediction
* Machine Learning Packages: https://scikit-learn.org/stable/


# Part I: Import and Inspect Data

In [1]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv('https://raw.githubusercontent.com/YiyangZhangyz738/Gas-Price/main/data/Processed_Data/GPR.csv')
df['Date'] = pd.to_datetime(df['Date'])
df['Rate'] = df['Rate'].astype(float)
df.head()

,Date,Rate
0,2015-07-30,0.0
1,2015-07-31,0.0
2,2015-08-01,0.0
3,2015-08-02,0.0
4,2015-08-03,0.0


# Part II: Prepare the Y varible for Regression

## 2.1. Write functions to calculte the Y variable for Regression

*(skip the step if the Y variable already exists)*

In [6]:

df['theta'] = (df['Date'].values.astype(np.int64) // 10 ** 9)/df['Rate']
df.head(20)

,Date,Rate,theta
0,2015-07-30,0.000000e+00,inf
1,2015-07-31,0.000000e+00,inf
2,2015-08-01,0.000000e+00,inf
3,2015-08-02,0.000000e+00,inf
4,2015-08-03,0.000000e+00,inf
5,2015-08-04,0.000000e+00,inf
6,2015-08-05,0.000000e+00,inf
7,2015-08-06,0.000000e+00,inf
8,2015-08-07,0.000000e+00,inf
9,2015-08-08,2.115954e+11,0.006801


## 2.2. Make Sure that the Data Type of Y is "numeric"

In [7]:
df.dtypes

Date     datetime64[ns]
Rate            float64
theta           float64
dtype: object

In [8]:
df['theta'] = pd.to_numeric(df['theta'])
df.dtypes

Date     datetime64[ns]
Rate            float64
theta           float64
dtype: object

# Part IV: Create the X variables

## 4.1. Shift the Y to get past values

reference:
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.shift.html

In [9]:
# generate a new variable as the previous 1 observable of your Y variable for regression
df['theta_past'] =df['theta'].shift(1)
df.head()

,Date,Rate,theta,theta_past
0,2015-07-30,0.0,inf,NaN
1,2015-07-31,0.0,inf,inf
2,2015-08-01,0.0,inf,inf
3,2015-08-02,0.0,inf,inf
4,2015-08-03,0.0,inf,inf


## 4.2. Calculate the Moving Averages

references:

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.rolling.html

https://towardsdatascience.com/moving-averages-in-python-16170e20f6c

In [10]:
#@title Define the Window
window = 10 #@param {type:"number"}


In [11]:
df['theta_past_ma10']=df['theta_past'].rolling(window=window,min_periods=1).mean()
df.head(20)

,Date,Rate,theta,theta_past,theta_past_ma10
0,2015-07-30,0.000000e+00,inf,NaN,NaN
1,2015-07-31,0.000000e+00,inf,inf,NaN
2,2015-08-01,0.000000e+00,inf,inf,NaN
3,2015-08-02,0.000000e+00,inf,inf,NaN
4,2015-08-03,0.000000e+00,inf,inf,NaN
5,2015-08-04,0.000000e+00,inf,inf,NaN
6,2015-08-05,0.000000e+00,inf,inf,NaN
7,2015-08-06,0.000000e+00,inf,inf,NaN
8,2015-08-07,0.000000e+00,inf,inf,NaN
9,2015-08-08,2.115954e+11,0.006801,inf,NaN


# Part V Train and Test Split

*reference*:

https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.TimeSeriesSplit.html

In [12]:
from sklearn.model_selection import TimeSeriesSplit
tss = TimeSeriesSplit()
print(tss)

TimeSeriesSplit(gap=0, max_train_size=None, n_splits=5, test_size=None)


In [13]:
# change the train and test split parameters
tss = TimeSeriesSplit(gap=0, max_train_size=None, n_splits=2, test_size=None)

In [14]:
for train_idx, test_idx in tss.split(df):
    print("TRAIN:", train_idx, "TEST:", test_idx)

TRAIN: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233
 234 235 236 237 238 239 240 241 242 243 244

In [15]:
train_idx

array([   0,    1,    2, ..., 1921, 1922, 1923])

In [16]:
test_idx

array([1924, 1925, 1926, 1927, 1928, 1929, 1930, 1931, 1932, 1933, 1934,
       1935, 1936, 1937, 1938, 1939, 1940, 1941, 1942, 1943, 1944, 1945,
       1946, 1947, 1948, 1949, 1950, 1951, 1952, 1953, 1954, 1955, 1956,
       1957, 1958, 1959, 1960, 1961, 1962, 1963, 1964, 1965, 1966, 1967,
       1968, 1969, 1970, 1971, 1972, 1973, 1974, 1975, 1976, 1977, 1978,
       1979, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989,
       1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000,
       2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011,
       2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022,
       2023, 2024, 2025, 2026, 2027, 2028, 2029, 2030, 2031, 2032, 2033,
       2034, 2035, 2036, 2037, 2038, 2039, 2040, 2041, 2042, 2043, 2044,
       2045, 2046, 2047, 2048, 2049, 2050, 2051, 2052, 2053, 2054, 2055,
       2056, 2057, 2058, 2059, 2060, 2061, 2062, 2063, 2064, 2065, 2066,
       2067, 2068, 2069, 2070, 2071, 2072, 2073, 20

In [17]:
train_df = df.filter(items=train_idx, axis=0)
test_df =  df.filter(items=test_idx, axis=0)

In [18]:
train_df.head()

,Date,Rate,theta,theta_past,theta_past_ma10
0,2015-07-30,0.0,inf,NaN,NaN
1,2015-07-31,0.0,inf,inf,NaN
2,2015-08-01,0.0,inf,inf,NaN
3,2015-08-02,0.0,inf,inf,NaN
4,2015-08-03,0.0,inf,inf,NaN


In [19]:
test_df.head()

,Date,Rate,theta,theta_past,theta_past_ma10
1924,2020-11-04,1.253192e+10,0.128029,0.126358,0.118644
1925,2020-11-05,1.283744e+10,0.124989,0.128029,0.119394
1926,2020-11-06,1.034641e+10,0.155090,0.124989,0.120933
1927,2020-11-07,9.591481e+09,0.167305,0.155090,0.125466
1928,2020-11-08,9.718210e+09,0.165133,0.167305,0.131241


# Part VI Prepare the Train and Test Data for Classification and Regression

## 6.2 Regression

### 6.2.1. Define the columns (Y, X) for Regression

In [20]:
cols_R = ['theta','theta_past_ma10']

### 6.2.2. Define the Data Frame of Train and Test Data for Regression

In [21]:
df_R_train = train_df[cols_R]
df_R_test = test_df[cols_R]

### 6.2.3. Export the Train and Test Data for Regression

In [22]:
df_R_train.head()

,theta,theta_past_ma10
0,inf,NaN
1,inf,NaN
2,inf,NaN
3,inf,NaN
4,inf,NaN


In [23]:
df_R_train.to_csv('Regression_Train.csv')

In [24]:
df_R_test.head()

,theta,theta_past_ma10
1924,0.128029,0.118644
1925,0.124989,0.119394
1926,0.155090,0.120933
1927,0.167305,0.125466
1928,0.165133,0.131241


In [25]:
df_R_test.to_csv('Regression_Test.csv')